## Part II: Predicting Hall of Fame Careers

In [3]:
# Connecting to SQLite Database
import pandas as pd
import sqlite3
conn = sqlite3.connect("lahman2016.sqlite")

In [4]:
queryMaster = 'select playerID,nameFirst,nameLast,bats,throws,debut,finalGame from Master;'
queryBatting = 'select * from Batting;'
queryPitching = 'select * from Pitching;'
queryFielding = 'select * from Fielding;'
queryHOF = 'select playerID,yearid from HallofFame where inducted == "Y" and category == "Player";'

In [5]:
master = conn.execute(queryMaster).fetchall()
master_df = pd.DataFrame(master)
batting = conn.execute(queryBatting).fetchall()
batting_df = pd.DataFrame(batting)
pitching = conn.execute(queryPitching).fetchall()
pitching_df = pd.DataFrame(pitching)
fielding = conn.execute(queryFielding).fetchall()
fielding_df = pd.DataFrame(fielding)
hof = conn.execute(queryHOF).fetchall()
hof_df = pd.DataFrame(hof)